In [1]:
import csv
import itertools

# Define well positions from A1 to A6
rows = "A"
cols = range(1, 7)
wells = [f"{row}{col}" for row, col in itertools.product(rows, cols)]

# Define parameters for Enzy type
hours_enzy = [0, 5, 20, 25, 45, 50, 65, 75, 92, 100, 115, 125, 140]
ph_values_enzy = [5, 6, 7, 8]
types_enzy = ["Enzy"]

# Collect Enzy data
data = []
for hour in hours_enzy:
    for well in wells:
        for t in types_enzy:
            for ph_value in ph_values_enzy:
                image_name = f"{well}_hr{hour}_{t.lower()}.JPG"
                data.append([well, hour, ph_value, t, image_name])

# Sort data before writing
#data.sort(key=lambda x: (x[0], x[1]))

# Write Enzy data to CSV (including header)
with open("well_data2.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["well_no", "hour", "pH_value", "type", "image_name"])  # Header
    writer.writerows(data)

print("CSV file 'well_data.csv' has been updated with Enzy data.")


# Define parameters for Hydro type
hours_hydro = [0, 5, 20, 25, 45, 50, 65, 75, 92, 100, 115, 125, 140]
ph_values_hydro = [7, 8]
types_hydro = ["Hydro"]

# Collect Hydro data
data = []
for hour in hours_hydro:
    for well in wells:
        for t in types_hydro:
            for ph_value in ph_values_hydro:
                image_name = f"{well}_hr{hour}_{t.lower()}.JPG"
                data.append([well, hour, ph_value, t, image_name])

# Sort data before appending
#data.sort(key=lambda x: (x[0], x[1]))

# Append Hydro data to CSV (without header)
with open("well_data2.csv", "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("CSV file 'well_data.csv' has been updated with Hydro data.")

# Define parameters for Hydro type
hours_hydro = [0, 5, 20, 25, 45, 50, 65, 75, 92, 100, 115, 125, 140]
ph_values_hydro = [5]
types_hydro = ["Hydro"]
rows = "H"
cols = range(7, 13)
wells = [f"{row}{col}" for row, col in itertools.product(rows, cols)]
# Collect Hydro data
data = []
for hour in hours_hydro:
    for well in wells:
        for t in types_hydro:
            for ph_value in ph_values_hydro:
                image_name = f"{well}_hr{hour}_{t.lower()}.JPG"
                data.append([well, hour, ph_value, t, image_name])

# Sort data before appending
#data.sort(key=lambda x: (x[0], x[1]))

# Append Hydro data to CSV (without header)
with open("well_data2.csv", "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("CSV file 'well_data.csv' has been updated with pH5 Hydro data.")

# Define parameters for Hydro type
hours_hydro = [0, 5, 20, 25, 45, 50, 65, 75, 92, 100, 115, 125, 140]
ph_values_hydro = [6]
types_hydro = ["Hydro"]
rows = "A"
cols = range(7, 13)
wells = [f"{row}{col}" for row, col in itertools.product(rows, cols)]
# Collect Hydro data
data = []
for hour in hours_hydro:
    for well in wells:
        for t in types_hydro:
            for ph_value in ph_values_hydro:
                image_name = f"{well}_hr{hour}_{t.lower()}.JPG"
                data.append([well, hour, ph_value, t, image_name])

# Sort data before appending
#data.sort(key=lambda x: (x[0], x[1]))

# Append Hydro data to CSV (without header)
with open("well_data2.csv", "a", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("CSV file 'well_data.csv' has been updated with pH6 Hydro data.")

CSV file 'well_data.csv' has been updated with Enzy data.
CSV file 'well_data.csv' has been updated with Hydro data.
CSV file 'well_data.csv' has been updated with pH5 Hydro data.
CSV file 'well_data.csv' has been updated with pH6 Hydro data.


In [2]:
import pandas as pd
import os

# Load the CSV file
csv_file_path = "well_data_updated.csv"
df = pd.read_csv(csv_file_path)

# Define the dataset root directory
dataset_root = "MLPR_Dataset_Copy"

# Function to find the correct image name based on well, hour, pH, and type
def get_correct_image_name(row):
    well_no = row["well_no"]  # e.g., A1, A2, ..., A12, H7, H8, ... H12
    hour = f"{row['hour']} hr"  # Format hour as folder name
    pH = f"pH{row['pH_value']}"
    exp_type = "Enzymatic" if "Enzy" in row["type"] else "Hydrolytic"
    
    # Construct the expected folder path
    subfolder = f"{pH} {exp_type}".strip()
    folder_path = os.path.join(dataset_root, hour, subfolder)
    
    if not os.path.exists(folder_path):
        return "IMAGE_NOT_FOUND"
    
    # Get all images in the folder (sorted to maintain order)
    image_files = sorted([img for img in os.listdir(folder_path) if img.lower().endswith(".jpg")])
    
    # Determine the well index
    if pH == "pH5" and exp_type == "Hydrolytic":
        well_index = int(well_no[1:]) - 7  # H7-H12 -> index 0-5
    elif pH == "pH6" and exp_type == "Hydrolytic":
        well_index = int(well_no[1:]) - 7  # A7-A12 -> index 0-5
    else:
        well_index = int(well_no[1:]) - 1  # Default case A1-A6 -> index 0-5
    
    if 0 <= well_index < len(image_files):
        return image_files[well_index]
    else:
        return "IMAGE_NOT_FOUND"

# Apply the function to update the image_name column
df["image_name"] = df.apply(get_correct_image_name, axis=1)

# Save the updated CSV file
updated_csv_path = "well_data_updated2.csv"
df.to_csv(updated_csv_path, index=False)

print(f"Updated CSV saved at: {updated_csv_path}")

FileNotFoundError: [Errno 2] No such file or directory: 'well_data_updated.csv'